---
order: 11
title: "5.11 THE CHAINS WE FORGED IN LIFE"
---

> EXERCISE 5.11: THE CHAINS WE FORGED IN LIFE
> 
> Modify the identity validation programs to support a chain of trust. 
> First, create some self signed certificates for the EA goverment (at 
> least two as described previously). The existing issuer script can already 
> do this. Just make the issuer private key for the self-signed certificate to 
> be the organization's own private key. Thus, the organization is signing 
> its own cert, and the private key used to sign the certificate matches the 
> public key _in_ the certificate. 
> 
> Next, create certificates for intermediate CAs such as "Department of Education", 
> "Department of Defense", "Espionage Agency" and so forth. These certificates should 
> be signed by the self signed certificates in the previous step. 
> 
> Finally, sign certificates for Alice, Bob, and Charlie by the espionage CA. Perhaps 
> create some certificates for employees of the defense department and the 
> education department. These certificates should be signed by the appropriate 
> intermediate CA. 
> 
> Now, modify the verification program to take a chain of certificates instead 
> of just a single certificate. Get rid of the command-line parameter for the 
> issuer's public key and instead hard-code which of the root certificate filenames 
> are trusted. To specify the chain of certificates, have the program take the 
> claimed identity as the first input (as it already does) and then an 
> arbitrary number of certificates to follow. Each certificate's issuer field 
> should indicate the next certificate in the chain. For example, to validate 
> Charlie, there may be three certificates: `charlie.cert`, `espionage.cert`, 
> `covert_root.cert`. The issuer of `charlie.cert` should have the same subject 
> name as `espionage.cert` and so forth. The verify program should only accept 
> an identity if the last certificate in the chain is already trusted. 

--------------------------------

## Creating two self-signed certificates

Let's call the keys: 

* `root1_private.key`, `root1_public.key`
* `root2_private.key`, `root2_public.key`

![generating keys](5_11_assets/generate_rsa_key_pairs.png)

Let's call the certificates: 

* root1.cert
* root2.cert

![creating the self signed root certificates](5_11_assets/creating-self-signed-certs.png)

The contents of the bash script above is given below: 

```bash
python3 fake_certs_issuer.py root1_private.key fake_cert_authority1 root1_public.key root1.cert
python3 fake_certs_issuer.py root2_private.key fake_cert_authority1 root2_public.key root2.cert
```

The usage of `fake_certs_issuer.py` script is: 

```
Usage: python3 fake_certs_issuer.py <issuer-private-key-file> <certificate-subject> <subject's-public-key-file> <certificate-output-file>
```